### Setup

First, please make sure you are using a GPU runtime to run this notebook, so inference is much faster. If the following command fails, use the `Runtime` menu above and select `Change runtime type`.

In [ ]:
!nvidia-smi

Next, you should install `diffusers` as well `scipy`, `ftfy` and `transformers`. `accelerate` is used to achieve much faster loading.

In [ ]:
!pip install --upgrade git+https://github.com/huggingface/diffusers.git
!pip install transformers scipy ftfy accelerate

In [ ]:
# install a checkpoint
import os
import subprocess
animemix = ['animemix_v3_offset.safetensors','https://civitai.com/api/download/models/60568']
animetarot = ['animetarotV51.safetensors','https://civitai.com/api/download/models/28609']
ck2use = animetarot
ckpt = ck2use[0]
!apt install axel
# !axel https://civitai.com/api/download/models/123021 --num-connections=20
if ckpt not in os.listdir() :
  subprocess.run(['axel', ck2use[1]])
os.listdir()

Now the stuff starts

In [ ]:
import torch
#from diffusers import StableDiffusionPipeline
from diffusers import DiffusionPipeline, EulerDiscreteScheduler, DPMSolverMultistepScheduler
#scheduler = EulerDiscreteScheduler.from_pretrained(repo_id, subfolder="scheduler")

#model_name = "google/ddpm-cifar10-32"
#model_name = "hakurei/waifu-diffusion" # waifu diffusion
#model_name = "./animemix_v3_offset.safetensor"
#model_name = "Linaqruf/animagine-xl" # idk
#model_name = "CompVis/stable-diffusion" # sd - 1.4
#model_name = "stabilityai/stable-diffusion-2-1"
#model_name= "runwayml/stable-diffusion-v1-5"



################[GOOD ONES]
model_name = "redstonehero/mixprov4_v4"    ####[ GOOD ANIME]########3
#model_name = "Yntec/epiCPhotoGasm"
#model_name = "nyxia/dynavision-xl"
#model_name ="stablediffusionapi/aniverse" #
#checkpoint_path="./123021"
#pipe = StableDiffusionPipeline.from_pretrained("hakurei/waifu-diffusion", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained(
    model_name, torch_dtype=torch.float16, safety_checker=None
)

pipe.to("cuda")


#pipe.load_lora_weights('.',weight_name=ckpt)
#pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config)
#pipe.unet.load_attn_procs("./animemix_v3_offset.safetensors")
#pipe = LayeredDiffusionPipeline().ConnectCkpt(model_name, checkpoint_path=model_path, vae_path=vae_path)
#pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
#pipe.unet.load_attn_procs(model_path)
pipe.enable_vae_tiling()
#pipe.enable_xformers_memory_efficient_attention()
#pipe.enable_sequential_cpu_offload()




text_encoder/model.safetensors not found


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Next, let's move the pipeline to GPU to have faster inference.

And we are ready to generate images:

In [ ]:
prompt = "a girl , short black hair ,  yellow jacket  , mountaintop, very sharp , rivers and waterfall in background"
import random
gen1 = random.randint(0,235346335233)
print("Seed :",gen1)
# Now to display an image you can either save it such as:
#image.save(f"astronaut_rides_horse.png")
negative_prompt = "EasyNegative, badhandv4, bad_prompt_version2-neg, verybadimagenegative_v1.3, sketch, painting , stray colors"
num_images = 4
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=1920,
    height=1080,
    num_inference_steps=60,
    guidance_scale=7.5,
    num_images_per_prompt=num_images,
    generator=torch.manual_seed(gen1),
).images
# or if you're in a google colab you can directly display it with

#grid = image_grid(image, rows=1, cols=num_images)


Seed : 141781950955


  0%|          | 0/60 [00:00<?, ?it/s]

In [ ]:
# I am not using this crap
'''
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid
'''


In [ ]:
from IPython.display import display
w, h = image[0].size
for i in image:
  display(i)


In [ ]:
# Just to reset cuda
torch.cuda.empty_cache()